In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pymongo import MongoClient

import os
import tqdm
import requests

from multiprocessing import Pool
import time
import itertools

In [3]:
from tqdm.notebook import tqdm

tqdm.pandas()

In [4]:
heroes = pd.DataFrame(requests.get(f"https://api.opendota.com/api/heroes").json())

In [5]:
heroes

,id,name,localized_name,primary_attr,attack_type,roles,legs
0,1,npc_dota_hero_antimage,Anti-Mage,agi,Melee,"[Carry, Escape, Nuker]",2
1,2,npc_dota_hero_axe,Axe,str,Melee,"[Initiator, Durable, Disabler, Jungler]",2
2,3,npc_dota_hero_bane,Bane,int,Ranged,"[Support, Disabler, Nuker, Durable]",4
3,4,npc_dota_hero_bloodseeker,Bloodseeker,agi,Melee,"[Carry, Disabler, Jungler, Nuker, Initiator]",2
4,5,npc_dota_hero_crystal_maiden,Crystal Maiden,int,Ranged,"[Support, Disabler, Nuker, Jungler]",2
...,...,...,...,...,...,...,...
114,120,npc_dota_hero_pangolier,Pangolier,agi,Melee,"[Carry, Nuker, Disabler, Durable, Escape, Init...",2
115,121,npc_dota_hero_grimstroke,Grimstroke,int,Ranged,"[Support, Nuker, Disabler, Escape]",2
116,126,npc_dota_hero_void_spirit,Void Spirit,int,Melee,"[Carry, Escape, Nuker, Disabler]",2
117,128,npc_dota_hero_snapfire,Snapfire,str,Ranged,"[Support, Nuker, Disabler, Escape]",2


In [6]:
synergy = pd.read_pickle("/mnt/b/Git_Projects/DOTA2Project/Stats_Data/synergy.pickle")

In [7]:
adversity = pd.read_pickle("/mnt/b/Git_Projects/DOTA2Project/Stats_Data/adversity.pickle")

In [8]:
all_ = pd.read_pickle("/mnt/b/Git_Projects/DOTA2Project/Stats_Data/data_new.pickle")

In [9]:
all_.head()

,_id,match_id,radiant_win,account_id0,hero_id0,account_id1,hero_id1,account_id2,hero_id2,account_id3,...,account_id129,hero_id129,account_id130,hero_id130,account_id131,hero_id131,account_id132,hero_id132,Radiant_heros,Dire_heros
0,5de94b5e279b6dc43f1378a3,2304329004,t,4294967295,14,4294967295,62,121397020,61,97735138,...,95882819,1,114997441,32,4294967295,35,4294967295,30,"[14, 62, 61, 18, 5]","[6, 1, 32, 35, 30]"
1,5de94b5e279b6dc43f1378a4,2304324185,t,124964100,26,4294967295,35,4294967295,18,169834263,...,176272692,12,4294967295,14,115451143,38,4294967295,80,"[26, 35, 18, 20, 74]","[7, 12, 14, 38, 80]"
2,5de94b5e279b6dc43f1378a5,2304335744,t,4294967295,29,4294967295,44,4294967295,73,4294967295,...,4294967295,77,108710635,39,137517574,109,158673305,58,"[29, 44, 73, 93, 27]","[75, 77, 39, 109, 58]"
3,5de94b5e279b6dc43f1378a6,2304333169,f,4294967295,93,4294967295,73,291017174,112,4294967295,...,199965965,1,4294967295,34,4294967295,14,279083894,71,"[93, 73, 112, 74, 53]","[104, 1, 34, 14, 71]"
4,5de94b5e279b6dc43f1378a7,2304332593,t,4294967295,22,132440989,100,4294967295,93,166469122,...,4294967295,33,124213727,68,4294967295,18,4294967295,39,"[22, 100, 93, 75, 94]","[80, 33, 68, 18, 39]"


In [10]:
from scipy import stats

Write a decorator function to apply the statistics to each of the score types (synergy, adversity etc..)

In [25]:
dict(heroes.id)[119]

KeyError: 119

In [28]:
adversity.loc[(1, 112)].values[0]

0.5207762557077625

In [32]:
import Data_Prep_Methods as dpm

dataset = dict()

for i, row in tqdm(list(all_.sample(100000).iterrows())):

    dataset[i] = {**dpm.synergy_scores(row),
                **dpm.adversity_scores(row),
                **dpm.hero_one_hot(row),
                **{"Radiant_win": row.radiant_win,
                    "match_id": row.match_id}}
    if i%10==0:
        dataframe = pd.DataFrame(dataset).T
        print(dataframe.shape)
        dataframe.to_pickle(os.getcwd() + "small_dataset_2.pickle")
    """
    try:
        dataset[i] = {**synergy_scores(row),
                      **adversity_scores(row),
                      **hero_one_hot(row),
                      **{"Radiant_win": row.radiant_win,
                         "match_id": row.match_id}}
        if i%10==0:
            dataframe = pd.DataFrame(dataset).T
            dataframe.to_pickle(os.getcwd() + "small_dataset_2.pickle")

    except:
        continue
    """
dataframe = pd.DataFrame(dataset).T

NameError: name 'synergy' is not defined

In [ ]:
dataset.to_pickle(os.getcwd() + "small_dataset_2.pickle")